In [1]:
from sqlalchemy import create_engine, exc
import pandas as pd
from utils import *
import datetime
import config
import time

In [2]:
## Glopal  variables
months = config.MONTHS
postgres = config.POSTGRS_CREDENTIALS
data_cols = config.DATA_COLS
dims = config.DWH_DIMS_FACTS

In [3]:
## utils
def select_dim(df, dims, dim_name):
    
    df_dim = df[dims[dim_name]]
    df_dim = df_dim.drop_duplicates()
    return df_dim


def convert_to_neumeric(df,cols):
    
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df[cols] = df[cols].fillna(0)
    return df


def convert_to_date(df,cols):
    
    default_date = pd.Timestamp('1900-01-01')
    for col in cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        df[col] = df[col].fillna(default_date)
    return df


def convert_to_string(df, cols):
    
    df[cols] = df[cols].fillna("N/A").astype(str)
    return df


In [4]:
def extract_data(data_cols, months):
    df_total = pd.DataFrame(columns=data_cols)
    
    for month in months:
        path = f"/home/jovyan/Data/{month}/listings (1).csv"
        df_source = pd.read_csv(path, low_memory=False)
        df_source = df_source[data_cols].dropna(how='all')
        df_total = pd.concat([df_total, df_source], axis=0, ignore_index=True)
        print(f">>>>>>> month {month} appended the dataframe")
        print(f"Now the dataframe shape is {df_total.shape}")

    return df_total

In [18]:
def transform_data(df):
    df["price"] = df[["price"]].replace({'\$': '', ',': ''}, regex=True)
    df = convert_to_neumeric(df, ["host_id","host_total_listings_count", "price", "accommodates", 
                                              "bathrooms", "bedrooms", "beds", "latitude",
                                              "longitude", "maximum_nights", "minimum_nights",
                                              "number_of_reviews","availability_30","availability_60",
                                              "availability_90","availability_365"])
    df = convert_to_string(df, ["host_name", "host_url", "host_response_rate",
                                            "host_verifications", "host_location", "neighbourhood",
                                            "region_name", "region_parent_name", 
                                            "region_parent_parent_name", "property_type", 
                                            "room_type"])
    
    df = convert_to_date(df, ["host_since", "last_scraped"])
    df["uid"] = range(1, len(df)+1)

    return df

In [19]:
def transform_host_dim(df, dims):
    dim_host = select_dim(df, dims, "dim_host")
    # dim_host_df = dim_host.dropna(subset= ["host_id"])
    dim_host = dim_host.rename(columns={"uid":"listing_uid"})
    return dim_host

In [20]:
def transform_location_dim(df, dims):
    dim_location_df = select_dim(df_total, dims, "dim_location")
    dim_location_df = dim_location_df.rename(columns={"uid":"listing_uid"})
    return dim_location_df

In [21]:
def transform_property_dim(df, dims):
    dim_property_df = select_dim(df, dims, "dim_property")
    dim_property_df = dim_property_df.rename(columns={"uid":"listing_uid"})
    return dim_property_df

In [22]:
def transform_room_dim(df, dims):
    dim_room_df = select_dim(df, dims, "dim_room")
    dim_room_df = dim_room_df[~dim_room_df['room_type'].str.contains(r'^\d+\.?\d*$', regex=True)].reset_index(drop = True)
    dim_room_df = dim_room_df.rename(columns={"uid":"listing_uid"})
    return dim_room_df


In [23]:
def extract_date_dim():
    path = f"/home/jovyan/Data/dimdates.csv"
    df_date = pd.read_csv(path, low_memory=False)
    dim_date_df = convert_to_date(df_date, ["Date"])
    dim_date_df = df_date[["Id", "Date", "DateShortDescription", "DayLongName", 
                              "MonthLongName","CalendarDayInMonth", "CalendarMonth",
                              "CalendarYear","CalendarQuarter","CalendarDay","CalendarWeek", "CalendarDayInWeek"]]
    dim_date_df.columns = [col.lower() for col in dim_date_df.columns]
    return dim_date_df

In [24]:
def extract_fact_listings(df, dims):
    df_fact = select_dim(df, dims, "fact_listings")
    df_fact = df_fact.rename(columns={"uid":"source_uid"})
    return df_fact 

## Data Extraction

In [25]:
df_total = extract_data(data_cols, months)

/tmp/ipykernel_23136/1804710694.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_total = pd.concat([df_total, df_source], axis=0, ignore_index=True)


>>>>>>> month jan appended the dataframe
Now the dataframe shape is (51221, 80)
>>>>>>> month March appended the dataframe
Now the dataframe shape is (96193, 80)
>>>>>>> month November appended the dataframe
Now the dataframe shape is (144409, 80)


## Data Transformation

In [26]:
df_transformed = transform_data(df_total)

In [27]:
host_dim = transform_host_dim(df_transformed, dims)

In [28]:
location_dim = transform_location_dim(df_transformed, dims)

In [29]:
property_dim = transform_property_dim(df_transformed, dims)

In [30]:
room_dim = transform_room_dim(df_transformed, dims)

In [31]:
date_dim = extract_date_dim()

In [32]:
fact_listings = extract_fact_listings(df_transformed, dims)

## Data Loading

In [33]:
engine = connect_to_db(postgres)

succefuly connected to airBnB_DWH at host: pgdatabase


In [38]:
write_df_to_db(fact_listings, "fact_listings", engine)

start writing data to fact_listings
Data of 144409 record Successfully written to the fact_listings table
the proccess toke 13.09923243522644 seconds


In [39]:
write_df_to_db(host_dim, "dim_host", engine)

start writing data to dim_host
Data of 144409 record Successfully written to the dim_host table
the proccess toke 19.022583484649658 seconds


In [40]:
write_df_to_db(location_dim, "dim_location", engine)

start writing data to dim_location
Data of 144409 record Successfully written to the dim_location table
the proccess toke 11.220252752304077 seconds


In [41]:
write_df_to_db(property_dim, "dim_property", engine)

start writing data to dim_property
Data of 144409 record Successfully written to the dim_property table
the proccess toke 3.2828822135925293 seconds


In [42]:
write_df_to_db(room_dim, "dim_room", engine)

start writing data to dim_room
Data of 144400 record Successfully written to the dim_room table
the proccess toke 3.323991298675537 seconds


In [57]:
finalize_things(engine)

engine disposed and connection closed successfully


In [35]:
fact_listings.dtypes

source_uid                    int64
price                       float64
maximum_nights              float64
minimum_nights              float64
number_of_reviews           float64
availability_30             float64
availability_60             float64
availability_90             float64
availability_365            float64
last_scraped         datetime64[ns]
dtype: object